# Hidden Markov Models

https://web.stanford.edu/~jurafsky/slp3/8.pdf


LATEX: http://csrgxtu.github.io/2015/03/20/Writing-Mathematic-Fomulars-in-Markdown/

## Markov Chains

<img src="./images/hmm1.png" width="70%" />

<img src="./images/hmm2.png" width="70%" />

**First-order Markow chain**: the probability of a particular state depends only on the previous state

**Markov Assumpation**:
\begin{align}
P(q_i|q_1...q_{i-1}) = P(qi|qi-1)
\end{align}

outgoing arcs from a given state must sum to 1:
\begin{align}
\sum_{j=1}^{n}a_{ij} = 1 \quad \forall i
\end{align}

### Alternative representation

Alternative representation for Markov chains that doesn't rely on a start and end state

<img src="./images/hmm4.png" width="70%"/>
<img src="./images/hmm3.png" width="70%"/>

The probability of state 1 being the first state can be represented either as $a_{01}$ or as $\pi_1$

Because $\pi_i$ expressed the probability $p(q_i|START)$, all the $\pi$ probabilities must sum to 1

\begin{align}
\sum_{i=1}^{n}\pi_i = 1
\end{align}


### Exercise
What are the probabilites of the following sequences?
* hot hot hot hot
* cold hot cold hot

In [36]:
# hot hot hot hot =
# P(H|START) * P(H|H) * P(H|H) * P(H|H) =
# 0.5 * 0.5 * 0.5 * 0.5
print 'probability of hot hot hot hot:', 0.5 * 0.5 * 0.5 * 0.5

# cold hot cold hot =
# P(C|START) * P(H|C) * P(C|H) * P(H|C) =
# 0.3 * 0.2 * 0.2 * 0.2
print 'probability of cold hot cold hot:', 0.3 * 0.2 * 0.2 * 0.2

probability of hot hot hot hot: 0.0625
probability of cold hot cold hot: 0.0024


## Hidden Markov Model

* Ovserved events
* Hidden events

<img src="./images/hmm5.png" width="70%"/>

**Markov Assumption**: $P(q_i|q_1...q_{i-1}) = P(qi|qi-1)$ <br>
The probability of a particular state depends only on the previous state

**Output Independence**: $P(o_i|q_1,...,q_T,o1,...,o_i,...o_T) = P(o_i|q_i)$ <br>
The probability of an output observation $o_i$ depends only on the state that produced the observation $q_i$ and not on any other states or any other observations

<img src="./images/hmm6.png"/>


**Fully connected / ergodic HMM**: have a (non-zero) probability of transition between any two states. 

**Left-to-right / Bakis HMM**: the state transitions proceed from left to right

<img src="./images/hmm7.png"/>
*Left-to-right hmm on the left. and fully connected HMM on the right*

## 3 fundamental problems

* **Problem 1 (Likelihood)**: Give an $HMM \lambda = (A,B)$ and an observation sequence $O$, determine the likelihood of $P(O|\lambda)$
* **Problem 2 (Decoding)**: Given an observation sequence $O$ and an $HMM \lambda = (A,B)$, discover the best hidden state sequence $Q$.
Determining which sequence of variables is the underlying source of some sequence of observations is called the decoding task

* **Problem 3 (Learning)**: Given an observation sequence $O$ and the set of states in the HMM, learn the HMM parameters A and B.

## Likelihood Computation: The Forward Algorithm

Give an $HMM \lambda = (A,B)$ and an observation sequence $O$, determine the likelihood of $P(O|\lambda)$

What is the probability of the sequence 3 1 3?

**Likelihood of the observation sequence**:
\begin{align}
P(O|Q) = \prod_{i=1}^{T}P(o_i|q_i)
\end{align}

<img src="./images/hmm8.png"/>

\begin{align}
P(313|hothotcold) = P(3|hot)×P(1|hot)×P(3|cold)
\end{align}

but since we don't know the weather states,

\begin{align}
P(O,Q) = P(O|Q) \times P(Q) = \prod_{i=1}^{n}P(o_i|q_i) \times \prod_{i=1}^{n}P(q_i|q_{i-1})
\end{align}

\begin{align}
P(313,hot\ hot\ cold) = P(hot|start)×P(hot|hot)×P(cold|hot) ×P(3|hot) × P(1|hot) × P(3|cold)
\end{align}

Now that we know how to compute the joint probability of the observations with a particular hidden state sequence, we can compute the total probability of the observations just by summing over all possible hidden state sequences:

\begin{align}
P(O) = \sum_{Q}P(O,Q) = \sum_{Q}P(O|Q)P(Q)
\end{align}

<img src="./images/hmm9.png"/>

\begin{align}
P(3 1 3) = P(3 1 3,cold\ cold\ cold)+P(3 1 3,cold\ cold\ hot)+P(3 1 3,hot\ hot\ cold)+...
\end{align}

<img src="./images/hmm6.png"/>

In [110]:
import itertools

HOT = 'HOT'
COLD = 'COLD'
END = 'END'

start = {
    'HOT': .8,
    'COLD': .2,
}

hot = {
    'HOT': .6,
    'COLD': .3,
    'END': .1,
    1: .2,
    2: .4,
    3: .4,
}

cold = {
    'HOT': .4,
    'COLD': .5,
    'END': .1,
    1: .5,
    2: .4,
    3: .1,
}

# P(313,hot hot cold)=P(hot|start)×P(hot|hot)×P(cold|hot)×P(3|hot)×P(1|hot)×P(3|cold)

def calc_weathers(weathers):
    p = 0
    for i, w in enumerate(weathers):
        if i == 0:
            p = start[w]
            last_w = w
            continue

        last_w = weathers[i-1]
        if last_w == HOT:
            p = p * hot[w]
        else:
            p = p * cold[w]

    return p

def calc_icecreams(weathers, icecreams):
    p = 0
    for w, ic in zip(weathers, icecreams):
        if p == 0:
            if w == COLD:
                p = cold[ic]
            else:
                p = hot[ic]
            continue
        if w == COLD:
            p = p * cold[ic]
        else:
            p = p * hot[ic]
    return p

def calc_weather_icecreams(weathers, icecreams):
    return calc_weathers(weathers) * calc_icecreams(weathers, icecreams)
    
def permutation(weathers):
    return list(set([p for p in itertools.product(weathers, repeat=len(weathers))]))

def get_p(weathers, icecreams):
    p = 0
    for combination in permutation(weathers):
        p += calc_weather_icecreams(combination, icecreams)

    return p



In [50]:
weathers = [COLD, HOT, HOT, COLD]
icecreams = [1, 2, 1, 3]

get_p(weathers, icecreams)

0.006380880000000001

## Forward Algorithm

* is a type of **dynamic programming**
* speed: $O(N^2T)$

<img src="./images/hmm10.png" width="70%"/>

Each cell of the forward algorithm trellis αt ( j) represents the probability of be- ing in state j after seeing the first t observations, given the automaton λ . The value of each cell αt ( j) is computed by summing over the probabilities of every path that could lead us to this cell. Formally, each cell expresses the following probability:

\begin{align}
\alpha_t(j) = P(o_1,o_2...o_t,q_t = j|\lambda)
\end{align}

Here, qt = j means “the $t$th state in the sequence of states is state $j$”. We compute this probability $αt(j)$ by summing over the extensions of all the paths that lead to the current cell. For a given state $q_j$ at time $t$, the value $αt(j)$ is computed as

\begin{align}
\alpha_t(j) = \sum_{i=1}^{N}\alpha_{t-1}(i)a_{ij}b_j(o_t)
\end{align}





<img src="./images/hmm11.png"/>

$\alpha_2(2)$: the forward probability of being at time step 2 in state 2 having generated the partial observation 3 1.


$\alpha_1(1) \times P(H|H) \times P(1|H)$ <br>
$\alpha_1(2) \times P(H|C) \times P(1|H)$




1. Initialization:
\begin{align}
a_1(j) = a_{0j}b_j(o_1) \quad 1 ≤ j ≤ N
\end{align}

2. Recursion(since states 0 and F are non-emitting):
\begin{align}
\alpha_t(j) = \sum_{i=1}^{N}\alpha_{t-1}(i)a_{ij}b_j(o_t); \quad 1 ≤ j ≤ N, 1 < t ≤ T
\end{align}

3. Termination:
\begin{align}
P(O|\lambda) = \alpha_(T)(q_F) = \sum_{i=1}^{N}\alpha_{T}(i)\alpha_{iF}
\end{align}

<img src="./images/hmm12.png"/>

<img src="./images/hmm13.png" width="70%"/>

In [118]:
O = [3, 1, 3]
T = len(O)
N = len([HOT, COLD])

matrix = []

for z in range(N):
    matrix.append([None] * T)

# initialiation step
for s in range(N):
    weather = HOT
    obs = hot
    if s == 1:
        weather = COLD
        obs = cold
    matrix[s][0] = start[weather] * obs[O[0]]


# recursion step
for t in range(1,T):
    for s in range(N):
        weather = HOT
        obs = hot
        if s == 1:
            weather = COLD
            obs = cold

        _sum = 0
        for sp in range(N):
            obs_sp = hot
            if sp == 1:
                obs_sp = cold
            
#             print t, matrix[sp][t-1], obs_sp[weather], obs[O[t]], weather
            _sum += matrix[sp][t-1] * obs_sp[weather] * obs[O[t]]
        matrix[s][t] = _sum


# termination step
_sum = 0
final = 0
for s in range(N):
    obs = hot
    if s == 1:
        obs = cold
    _sum += matrix[s][T-1] * obs[END]

final = _sum
    
print matrix
print final

[[0.32000000000000006, 0.04000000000000001, 0.018080000000000006], [0.020000000000000004, 0.053000000000000005, 0.003850000000000001]]
0.002193


In [117]:
# P(3|H) = 
print hot[3]
# P(3|C)
print cold[3]

print 0.040 * hot[HOT] * hot[3] +  0.053 * cold[HOT] * hot[3]
print 0.040 * 0.6 * 0.4  +  0.053 * 0.4 * 0.4

print 0.040 * hot[COLD] * cold[3] +  0.053 * cold[COLD] * cold[3]
print 0.040 * 0.3 * 0.1  +  0.053 * 0.5 * 0.1

# final
print 0.01808 * 0.1 + 0.00385 * 0.1

0.4
0.1
0.01808
0.01808
0.00385
0.00385
0.002193


## Decoding: The Viterbi Algorithm
Given a sequence of ice-cream observations 3 1 3 and an HMM, the task of the decoder is to find the best hidden weather sequence (H H H)

the Viterbi algorithm is identical to the forward algorithm except that it takes the max over the previous path probabilities whereas the forward algorithm takes the sum. Note also that the Viterbi algorithm has one component that the forward algorithm doesn’t have: backpointers. The reason is that while the forward algorithm needs to pro- duce an observation likelihood, the Viterbi algorithm must produce a probability and also the most likely state sequence

In [130]:
O = [3, 1, 3]
T = len(O)
N = len([HOT, COLD])

matrix = []
backpointer = []

for z in range(N):
    matrix.append([None] * T)
    backpointer.append([None] * T)

# initialiation step
for s in range(N):
    weather = HOT
    obs = hot
    if s == 1:
        weather = COLD
        obs = cold
    matrix[s][0] = start[weather] * obs[O[0]]
    backpointer[s][0] = 0


# recursion step
for t in range(1,T):
    for s in range(N):
        weather = HOT
        obs = hot
        if s == 1:
            weather = COLD
            obs = cold

        _values = []
        _bvalues = []
        for sp in range(N):
            obs_sp = hot
            if sp == 1:
                obs_sp = cold
            
#             print t, matrix[sp][t-1], obs_sp[weather], obs[O[t]], weather
            _values.append(matrix[sp][t-1] * obs_sp[weather] * obs[O[t]])
            _bvalues.append(matrix[sp][t-1] * obs_sp[weather] * obs[O[t]])
        matrix[s][t] = max(_values)
        backpointer[s][t] = np.argmax(_bvalues)


# termination step
_values = []
_bvalues = []
final = 0
finalb = 0
for s in range(N):
    obs = hot
    if s == 1:
        obs = cold
    _values.append(matrix[s][T-1] * obs[END])
    _bvalues.append(matrix[s][T-1] * obs[END])

final = max(_values)
finalb = np.argmax(_bvalues)
    
print matrix
print _bvalues #is this right?
print final
print finalb # is this right?

[[0.32000000000000006, 0.03840000000000001, 0.009216000000000002], [0.020000000000000004, 0.04800000000000001, 0.0024000000000000007]]
[0.0009216000000000003, 0.0002400000000000001]
0.0009216
0
